Q1: What's the name of the output?

In [1]:
import onnxruntime as ort

session = ort.InferenceSession(
    "hair_classifier_v1.onnx", providers=["CPUExecutionProvider"]
)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

print(f"Input: {input_name}, Output: {output_name}")


Input: input, Output: output


Q2: Based on the previous homework, what should be the target size for the image?

In [2]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST) # type: ignore
    return img

img = download_image("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg")
X = prepare_image(img, target_size=(200, 200))
print(X.size)


(200, 200)


Q3: After the pre-processing, what's the value in the first pixel, the R channel?

In [7]:
import numpy as np

def pytorch_preprocessing(X):
  X = X / 255.

  mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
  std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

  X = X.transpose(0, 3, 1, 2)
  X = (X - mean) / std

  return X.astype(np.float32)

def preprocess(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')

    small = img.resize(target_size, Image.NEAREST) # type: ignore
    x = np.array(small, dtype='float32')
    batch = np.expand_dims(x, axis=0)

    return pytorch_preprocessing(batch)

In [11]:
img = download_image("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg")
X = preprocess(img, target_size=(200, 200))
print(X[0][0][0])

[-1.0732939  -1.0047948  -1.0390444  -1.1075435  -1.0904187  -1.0219196
 -1.0390444  -1.1246682  -1.0219196  -1.0561692  -1.0390444  -1.0047948
 -1.0904187  -1.0390444  -1.0561692  -1.1246682  -1.1589177  -1.1589177
 -1.1931672  -1.2274168  -1.2445415  -1.3301653  -1.2445415  -1.2274168
 -1.2959157  -1.2274168  -1.2445415  -1.210292   -1.141793   -1.1246682
 -1.0732939  -1.0732939  -0.9876701  -0.9534206  -0.93629587 -0.86779684
 -0.93629587 -0.6965493  -0.7992978  -0.6280503  -0.5253018  -0.5253018
 -0.42255327 -0.30268002 -0.19993149 -0.08005822 -0.18280673 -0.08005822
  0.22818747  0.29668647  0.467934    0.8618033   1.3070469   1.5296687
  1.8379142   1.992037    1.9064132   1.9406627   1.992037    2.0262866
  2.14616     2.060536    1.7694153   1.5467935   1.5639182   1.8892885
  2.0434113   2.1119103   2.0434113   2.0434113   1.9749123   1.8892885
  1.5639182   0.74193007  0.34806073  0.05693981  0.51930827  0.19393796
  0.8275538   1.4611697   1.5296687   1.2385479   0.5878073  

Q4: What's the output of the model?

In [15]:
result = session.run([output_name], {input_name: X})
preds = result[0][0].tolist()
preds[0]

0.09156627207994461

In [18]:
import onnxruntime as ort
import numpy as np

from io import BytesIO
from urllib import request

from PIL import Image


session = ort.InferenceSession(
    "hair_classifier_v1.onnx", providers=["CPUExecutionProvider"]
)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def pytorch_preprocessing(X):
  X = X / 255.

  mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
  std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

  X = X.transpose(0, 3, 1, 2)
  X = (X - mean) / std

  return X.astype(np.float32)


def preprocess(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')

    small = img.resize(target_size, Image.NEAREST) # type: ignore
    x = np.array(small, dtype='float32')
    batch = np.expand_dims(x, axis=0)

    return pytorch_preprocessing(batch)


def predict(url):
    img = download_image(url)
    X = preprocess(img, target_size=(200, 200))
    
    result = session.run([output_name], {input_name: X})
    preds = result[0][0].tolist()

    return preds


def lambda_handler(event, context):
    url = event["url"]
    result = predict(url)
    
    return result


In [19]:
predict("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg")

[0.09156627207994461]